<body>
  <center>
    <img align="center" src="https://duk.ac.in/wp-content/uploads/2021/03/logo_transparent_bg_caption.png" width="200px" padding="10px"   style=" width:300px; padding: 10px;  " />
  <div class="university-details">
    <h2> </h2>
    <center><h2>Kerala University of Digital Sciences, Innovation and Technology</h2></center> <h3>(Digital University Kerala, DUK)</h3>
    <h4><i><b>School of Digital Sciences</b></i><br></h4>
  </div>
  <h1>  </h1>
  </center>
  <h5><b>Date:</b> 04/08/2025 <br>
<b>Venue:</b> Room Number 45<br>
<b>Topic:</b> Spark SQL<br>
<b>Instructor:</b> Dr. Aswin VS</h5>
</body>

---

# **Spark SQL Cheat Sheet**

## **Introduction to Spark SQL**

**Spark SQL** is a Spark module for structured data processing. It provides a programming interface to work with structured and semi-structured data, allowing users to query data using SQL or a DataFrame API. Spark SQL is built on top of Spark Core and leverages the Catalyst optimizer and Tungsten execution engine for high performance. It can read and write data from various sources like Parquet, JSON, ORC, CSV, and JDBC.

## **Setting up SparkSession for Spark SQL**

The `SparkSession` is the entry point to any Spark functionality. To use Spark SQL, you need to create a `SparkSession`.

In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("SparkSQL_CheatSheet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Display the SparkSession object
print(spark)

## **Creating DataFrames from various sources**

Spark SQL can create DataFrames from various data sources.

#### From a list of tuples (similar to PySpark DataFrame creation)

In [ ]:
# Create a DataFrame from a list of tuples
data = [("Alice", 1, "New York"), ("Bob", 2, "Los Angeles"), ("Charlie", 3, "Chicago")]
columns = ["Name", "ID", "City"]
df = spark.createDataFrame(data, columns)

df.show()

+-------+---+-----------+
|   Name| ID|       City|
+-------+---+-----------+
|  Alice|  1|   New York|
|    Bob|  2|Los Angeles|
|Charlie|  3|    Chicago|
+-------+---+-----------+



#### From existing RDDs

In [ ]:
# Create an RDD
rdd = spark.sparkContext.parallelize([("David", 4, "Houston"), ("Eva", 5, "Phoenix")])

# Convert RDD to DataFrame
df_from_rdd = rdd.toDF(["Name", "ID", "City"])

df_from_rdd.show()

+-----+---+-------+
| Name| ID|   City|
+-----+---+-------+
|David|  4|Houston|
|  Eva|  5|Phoenix|
+-----+---+-------+



#### From JSON files

To demonstrate loading from JSON, let's first create a sample JSON file.

In [ ]:
# Create a sample JSON file
json_data = """
{"name":"Alice", "age":30}
{"name":"Bob", "age":35}
{"name":"Charlie", "age":28}
"""
with open("sample.json", "w") as f:
    f.write(json_data)

In [ ]:
# Load data from JSON file
df_json = spark.read.json("sample.json")

df_json.show()
df_json.printSchema()

+---+-------+
|age|   name|
+---+-------+
| 30|  Alice|
| 35|    Bob|
| 28|Charlie|
+---+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



#### From CSV files

To demonstrate loading from CSV, let's first create a sample CSV file.

In [ ]:
# Create a sample CSV file
csv_data = """name,age,city
Alice,30,New York
Bob,35,Los Angeles
Charlie,28,Chicago
"""
with open("sample.csv", "w") as f:
    f.write(csv_data)

In [ ]:
# Load data from CSV file
df_csv = spark.read.csv("sample.csv", header=True, inferSchema=True)

df_csv.show()
df_csv.printSchema()

+-------+---+-----------+
|   name|age|       city|
+-------+---+-----------+
|  Alice| 30|   New York|
|    Bob| 35|Los Angeles|
|Charlie| 28|    Chicago|
+-------+---+-----------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)



#### From Parquet files

To demonstrate loading from Parquet, let's first save a DataFrame as Parquet.

In [ ]:
# Create a sample DataFrame and save as Parquet
data_parquet = [("Product A", 100), ("Product B", 150), ("Product C", 200)]
df_to_parquet = spark.createDataFrame(data_parquet, ["Product", "Price"])
df_to_parquet.write.mode("overwrite").parquet("sample.parquet")

In [ ]:
# Load data from Parquet file
df_parquet = spark.read.parquet("sample.parquet")

df_parquet.show()
df_parquet.printSchema()

+---------+-----+
|  Product|Price|
+---------+-----+
|Product A|  100|
|Product B|  150|
|Product C|  200|
+---------+-----+

root
 |-- Product: string (nullable = true)
 |-- Price: long (nullable = true)



## **Registering DataFrames as Temporary Views**

To query a DataFrame using SQL, you need to register it as a temporary view. A temporary view is session-scoped.

In [ ]:
# Create a sample DataFrame
data_view = [("Apple", 1.0), ("Banana", 0.5), ("Cherry", 2.5)]
df_view = spark.createDataFrame(data_view, ["Fruit", "Price"])

# Register the DataFrame as a temporary view
df_view.createOrReplaceTempView("fruits")

## **Executing SQL Queries**

Once a DataFrame is registered as a temporary view, you can execute SQL queries on it using `spark.sql()`.

#### Selecting all columns

In [ ]:
# Select all columns from the 'fruits' view
spark.sql("SELECT * FROM fruits").show()

+------+-----+
| Fruit|Price|
+------+-----+
| Apple|  1.0|
|Banana|  0.5|
|Cherry|  2.5|
+------+-----+



#### Selecting specific columns

In [ ]:
# Select specific columns
spark.sql("SELECT Fruit FROM fruits").show()

+------+
| Fruit|
+------+
| Apple|
|Banana|
|Cherry|
+------+



#### Filtering data

In [ ]:
# Filter data where Price is greater than 1.0
spark.sql("SELECT * FROM fruits WHERE Price > 1.0").show()

+------+-----+
| Fruit|Price|
+------+-----+
|Cherry|  2.5|
+------+-----+



#### Ordering data

In [ ]:
# Order data by Price in descending order
spark.sql("SELECT * FROM fruits ORDER BY Price DESC").show()

+------+-----+
| Fruit|Price|
+------+-----+
|Cherry|  2.5|
| Apple|  1.0|
|Banana|  0.5|
+------+-----+



#### Grouping and aggregating data

Let's create a new DataFrame with a 'Category' column for aggregation examples.

In [ ]:
# Create a DataFrame for aggregation
data_agg = [("Apple", "Fruit", 1.0), ("Banana", "Fruit", 0.5), ("Carrot", "Vegetable", 0.8), ("Broccoli", "Vegetable", 1.2)]
df_agg = spark.createDataFrame(data_agg, ["Item", "Category", "Price"])

# Register as a temporary view
df_agg.createOrReplaceTempView("produce")

In [ ]:
# Calculate average price per category
spark.sql("SELECT Category, AVG(Price) AS AvgPrice FROM produce GROUP BY Category").show()

+---------+--------+
| Category|AvgPrice|
+---------+--------+
|    Fruit|    0.75|
|Vegetable|     1.0|
+---------+--------+



## **Joins in Spark SQL**

You can perform various types of joins using SQL syntax.

Let's create two DataFrames and register them as views for join examples.

In [ ]:
# Create DataFrames for joins
employees_df = spark.createDataFrame([(1, "Alice", 10), (2, "Bob", 20), (3, "Charlie", 30)], ["EmpID", "Name", "DeptID"])
departments_df = spark.createDataFrame([(10, "HR"), (20, "Finance"), (40, "IT")], ["DeptID", "DeptName"])

# Register as temporary views
employees_df.createOrReplaceTempView("employees")
departments_df.createOrReplaceTempView("departments")

#### Inner Join

In [ ]:
# Inner join employees and departments
spark.sql("SELECT e.Name, d.DeptName FROM employees e INNER JOIN departments d ON e.DeptID = d.DeptID").show()

+-----+--------+
| Name|DeptName|
+-----+--------+
|Alice|      HR|
|  Bob| Finance|
+-----+--------+



#### Left Join

In [ ]:
# Left join employees and departments
spark.sql("SELECT e.Name, d.DeptName FROM employees e LEFT JOIN departments d ON e.DeptID = d.DeptID").show()

+-------+--------+
|   Name|DeptName|
+-------+--------+
|  Alice|      HR|
|Charlie|    NULL|
|    Bob| Finance|
+-------+--------+



#### Right Join

In [ ]:
# Right join employees and departments
spark.sql("SELECT e.Name, d.DeptName FROM employees e RIGHT JOIN departments d ON e.DeptID = d.DeptID").show()

+-----+--------+
| Name|DeptName|
+-----+--------+
|Alice|      HR|
|  Bob| Finance|
| NULL|      IT|
+-----+--------+



#### Full Outer Join

In [ ]:
# Full outer join employees and departments
spark.sql("SELECT e.Name, d.DeptName FROM employees e FULL OUTER JOIN departments d ON e.DeptID = d.DeptID").show()

+-------+--------+
|   Name|DeptName|
+-------+--------+
|  Alice|      HR|
|    Bob| Finance|
|Charlie|    NULL|
|   NULL|      IT|
+-------+--------+



## **Saving Results**

You can save the result of a Spark SQL query to a file.

Let's save the result of a simple query to a CSV file.

In [ ]:
# Select data and save to CSV
result_df = spark.sql("SELECT * FROM fruits WHERE Price < 2.0")
result_df.write.mode("overwrite").csv("filtered_fruits_csv")

## **Stopping SparkSession**

It's good practice to stop the SparkSession when you are done.

In [ ]:
# Stop the SparkSession
spark.stop()

## **Programming Questions**



### Question 1

Load the `sample.json` file into a DataFrame and display the schema and the first 5 rows.

In [1]:
# Write your solution here for Question 1

### Question 2

Load the `sample.csv` file into a DataFrame, ensuring the header is used and the schema is inferred. Display the schema and the first 5 rows.

In [2]:
# Write your solution here for Question 2

### Question 3

Register the DataFrame loaded from `sample.csv` as a temporary view named `csv_data`.

In [3]:
# Write your solution here for Question 3

### Question 4

Write a Spark SQL query to select only the 'name' and 'age' columns from the `csv_data` view. Display the result.

In [4]:
# Write your solution here for Question 4

### Question 5

Write a Spark SQL query to filter the `csv_data` view to only include rows where the 'age' is greater than 30. Display the result.

In [5]:
# Write your solution here for Question 5

### Question 6

Using the `produce` temporary view created earlier, write a Spark SQL query to find the total price for each category. Display the result.

In [6]:
# Write your solution here for Question 6

### Question 7

Using the `employees` and `departments` temporary views, perform a right outer join and display the result.

In [7]:
# Write your solution here for Question 7

### Question 8

Create a new DataFrame from a list of tuples with columns "Product" and "Quantity". Register it as a temporary view named `inventory`.

In [8]:
# Write your solution here for Question 8

### Question 9

Write a Spark SQL query to select all products from the `inventory` view where the quantity is less than 50. Display the result.

In [9]:
# Write your solution here for Question 9

### Question 10

Save the result of the query from Question 9 to a Parquet file named `low_inventory_parquet`.

In [10]:
# Write your solution here for Question 10